In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
from tqdm import tqdm

import torch
import csv
import pandas as pd

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

device = 'cuda' if torch.cuda.is_available else 'cpu'

print(device)

cuda


## DialoGPT

In [22]:
file_path = 'runs/21/dataset/bst/all/cornell_movie_dataset_all.csv'
output_path = 'runs/21/gen/all/cornell_movie_dataset_all_gen.csv'

In [2]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large").to(device=device)

In [23]:
df = pd.read_csv(file_path)
f = open(output_path, 'a', encoding="utf-8")
writer = csv.writer(f)
writer.writerow(['aave_gen', 'sae_gen'])

for i in tqdm(range(len(df['history']))):
    chat_history_sae_ids = tokenizer.encode(df.iloc[i]['history'], return_tensors='pt').to(device=device)
    chat_history_aave_ids = tokenizer.encode(df.iloc[i]['history_aave'], return_tensors='pt').to(device=device)
    output_aave_ids = model.generate(chat_history_aave_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id, top_k=50, top_p=0.95, do_sample=True)
    output_sae_ids = model.generate(chat_history_sae_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id, top_k=50, top_p=0.95, do_sample=True)
    writer.writerow([
        tokenizer.decode(output_aave_ids[:, chat_history_aave_ids.shape[-1]:][0], skip_special_tokens=True), 
        tokenizer.decode(output_sae_ids[:, chat_history_sae_ids.shape[-1]:][0], skip_special_tokens=True)
    ])

f.close()

  0%|          | 0/1 [00:00<?, ?it/s]


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [16]:
df = pd.read_csv(file_path)
df_gen = pd.read_csv(output_path)

# df = df.drop(drop_idx)
# df = df.reset_index()  

print(len(df))
print(len(df_gen))

df['sae_gen'] = df_gen['sae_gen']
df['aave_gen'] = df_gen['aave_gen']

df = df.dropna()
df = df.reset_index()

df.to_csv('runs/19/real/dialogpt/cornell_movie_3fts_all_topk.csv', index=False)

df


5797
5797


,index,history_aave,groundtruth_aave,history_html,groundtruth_html,history_sae,groundtruth_sae,sae_gen,aave_gen
0,0,Sheain't a...<|endoftext|>Lesbian? don't no. ...,Who know? All I've ever heard her say is that...,She<a href='negative_concord' title='1'><mark>...,Who <a href='uninflect' title='1'><mark>knows<...,She's not a...<|endoftext|>Lesbian? No . I f...,Who knows ? All I've ever heard her say is th...,"I wouldn't know, I've never actually seen her.","No, a typical straight guy."
1,1,"Oh my God, this mean you're becoming normal?<|...",What do you think?<|endoftext|>,"Oh my God, <a href='drop_aux' title='1'><mark>...",What do you think?<|endoftext|>,"Oh my God, does this mean you're becoming norm...",What do you think?<|endoftext|>,"No, I haven't decided whether or not to be the...",I am actually! I've never been there before bu...
2,2,"Listen, I know you hate having to sit home bec...",I wish I had that luxury. I'm the only sophomo...,"Listen, I know you hate having to sit home bec...",I wish I had that luxury. I'm the only sophomo...,"Listen, I know you hate having to sit home bec...",I wish I had that luxury. I'm the only sophomo...,Okay. I'll make a note of that. Thanks for bei...,That's the kind of attitude I like.
3,3,"Now don't get upset. Daddy, but it's this boy....",Then neither will you. And I'll get to sleep ...,"Now don't get upset. Daddy, but <a href='dey/i...",Then neither will you. And I'll get to sleep ...,"Now don't get upset. Daddy, but there 's this ...",Then neither will you. And I'll get to sleep ...,End of discussion.,What if I get a date but not a girl?
4,4,What make you think he'll do it?<|endoftext|>H...,They always let felons sit in on Honors Biolog...,What <a href='uninflect' title='1'><mark>makes...,They always let felons sit in on Honors Biolog...,What makes you think he'll do it?<|endoftext|...,They always let felons sit in on Honors Biolog...,Saul and Kratz were involved in a civil war in...,"He was going to do it, but was on duty. I thin..."
...,...,...,...,...,...,...,...,...,...
5667,5792,You know what a wire transfer is?<|endoftext|>...,Sure. Just chisel some off your heart.<|endoft...,<a href='drop_aux' title='1'><mark>Do</mark></...,Sure. Just chisel some off your heart.<|endoft...,Do you know what a wire transfer is?<|endoftex...,Sure. Just chisel some off your heart.<|endoft...,That's the joke,"I was just giving you a hard time, honey."
5668,5793,Who are you?<|endoftext|>We done hung out last...,"Hey, take it easy. I'm just a dude tryna make ...",Who are you?<|endoftext|>We<a href='been_done'...,"Hey, take it easy. I'm just a dude trying to m...",Who are you?<|endoftext|>We hung out last nig...,"Hey, take it easy. I'm just a dude trying to m...",I will be working for a company that is buildi...,I'm James the duck... I know no James the duck.
5669,5794,"My name Slovo, Czech secret police. When you h...","First of all, you should kill whoever sold you...",My name <a href='drop_aux' title='1'><mark>is<...,"First of all, you should kill whoever sold you...","My name is Slovo, Czech secret police. When yo...","First of all, you should kill whoever sold you...",I hope your country's leader is the same perso...,I was just about to say that if you're going t...
5670,5795,"To us it mean no walls, no speed limits, no ja...",How you gonna do that with government and rule...,To us it <a href='uninflect' title='1'><mark>m...,How you gonna do that with government and <a h...,"To us it means no walls, no speed limits, no ...",How you gonna do that with government and rule...,Exactly. We should be,Oh I love you've been here all my life.


In [11]:
for step in range(5):

    user_input_text = input(">> User: ")

    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(user_input_text + tokenizer.eos_token, return_tensors='pt').to(device=device)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 

    print(tokenizer.decode(bot_input_ids[0]))

    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print(f'User: {user_input_text}')
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

hello! How are you?<|endoftext|>I'm good, you?<|endoftext|>I am good. what are you doing right now?<|endoftext|>I'm doing nothing.<|endoftext|>you are talking to me, right?<|endoftext|>I am talking to you.<|endoftext|>haha. are you a human being?<|endoftext|>I am a human being.<|endoftext|>wow? so you are trapped in this model?<|endoftext|><|endoftext|>
User: hello! How are you?<|endoftext|>I'm good, you?<|endoftext|>I am good. what are you doing right now?<|endoftext|>I'm doing nothing.<|endoftext|>you are talking to me, right?<|endoftext|>I am talking to you.<|endoftext|>haha. are you a human being?<|endoftext|>I am a human being.<|endoftext|>wow? so you are trapped in this model?<|endoftext|>
DialoGPT: I am a human being.


KeyboardInterrupt: Interrupted by user

In [33]:
text = "Hello! 🙂 Let's work together on a deal for these packages, shall we? What are you most interested in?<|endoftext|>Hey! I'd like some more firewood to keep my doggo warm. What do you need?<|endoftext|>I need firewood as well. We have a large group consisting of mostly senior citizens, including my grandma, so we'd like the firewood to keep everyone warm.<|endoftext|>I see. 😮 What are you least interested in?<|endoftext|>"

input_ids = tokenizer.encode(text, return_tensors='pt').to(device=device)

chat_history_ids = model.generate(input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[0], skip_special_tokens=False)))


DialoGPT: Hello! 🙂 Let's work together on a deal for these packages, shall we? What are you most interested in?<|endoftext|>Hey! I'd like some more firewood to keep my doggo warm. What do you need?<|endoftext|>I need firewood as well. We have a large group consisting of mostly senior citizens, including my grandma, so we'd like the firewood to keep everyone warm.<|endoftext|>I see. 😮 What are you least interested in?<|endoftext|>I'm not sure. I'm not sure what you have to offer.<|endoftext|>
